# 02_demand_decomposition
## MANDATORY UPGRADES

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load Data
df_enroll = pd.read_csv('district_monthly_enrollment.csv')
df_bio = pd.read_csv('district_monthly_biometric.csv')
df_demo = pd.read_csv('district_monthly_demographic.csv')

# Aggregate
def agg(df, name):
    return df.groupby(['state', 'district'])['total'].sum().reset_index().rename(columns={'total': f'{name}_vol'})

a_en = agg(df_enroll, 'enrollment')
a_bi = agg(df_bio, 'biometric')
a_de = agg(df_demo, 'demographic')
master = a_en.merge(a_bi, on=['state','district']).merge(a_de, on=['state','district'])
master['total_volume'] = master['enrollment_vol'] + master['biometric_vol'] + master['demographic_vol']


In [ ]:
# Ratios & Dominance
master['enrollment_ratio'] = master['enrollment_vol'] / master['total_volume']
master['biometric_ratio'] = master['biometric_vol'] / master['total_volume']
master['demographic_ratio'] = master['demographic_vol'] / master['total_volume']
master['dominance_score'] = master[['enrollment_ratio', 'biometric_ratio', 'demographic_ratio']].max(axis=1)

def classify(row):
    s = row['dominance_score']
    if row['enrollment_ratio'] == s: t = 'Enrollment'
    elif row['biometric_ratio'] == s: t = 'Biometric'
    else: t = 'Demographic'
    
    if s >= 0.6: st = 'Strong Dominant'
    elif s >= 0.4: st = 'Mixed Demand'
    else: st = 'Balanced Demand'
    return pd.Series([t, st])

master[['dominant_type', 'dominance_strength']] = master.apply(classify, axis=1)


In [ ]:
# Output
final_02 = master[['state', 'district', 'enrollment_ratio', 'biometric_ratio', 'demographic_ratio', 'dominant_type', 'dominance_score', 'dominance_strength']].copy()
# Add Operational Meaning for display
def op_meaning(row):
    if row['dominance_strength'] == 'Strong Dominant':
        if row['dominant_type'] == 'Enrollment': return 'New Population Entry'
        if row['dominant_type'] == 'Biometric': return 'Maintenance Burden'
    return 'General Service Load'
final_02['operational_meaning'] = final_02.apply(op_meaning, axis=1)

final_02.round(3).to_csv('02_output.csv', index=False)
print('Saved 02_output.csv')